# Flaw Engine: ROBUST Cloud Trainer ♟️🚀

### 🏁 Step 1: Clean Clone
This wipes everything and pulls a fresh, light copy of the code.

In [ ]:
import os
URL = "https://github.com/Gideon-gg101/flaw-engine.git"
ROOT = "/content/flaw_worker"
!rm -rf {ROOT}
!git clone --depth 1 {URL} {ROOT}
if os.path.exists(ROOT): 
    print("\n✅ CLONE SUCCESS!")
    !ls -F {ROOT}

### 🏁 Step 2: Build
Compiles the C++ core.

In [ ]:
import os, shutil, glob
!apt install -y cmake g++ > /dev/null

# RECURSIVE SEARCH FOR CORE
CORE_DIR = None
for r, dirs, files in os.walk('/content/flaw_worker'):
    if 'CMakeLists.txt' in files and 'board.cpp' in files:
        CORE_DIR = r
        break

if not CORE_DIR:
    raise Exception("❌ Could not find C++ core directory!")

print(f"Targeting: {CORE_DIR}")
BUILD_DIR = os.path.join(CORE_DIR, "build")
if os.path.exists(BUILD_DIR): shutil.rmtree(BUILD_DIR)
os.makedirs(BUILD_DIR, exist_ok=True)

%cd {BUILD_DIR}
!cmake ..
!make -j$(nproc)

# Deploy to AI folder (find it too)
AI_DIR = None
for r, dirs, files in os.walk('/content/flaw_worker'):
    if 'selfplay_worker.py' in files:
        AI_DIR = r
        break

libs = glob.glob("flaw_core*.so")
if libs and AI_DIR:
    lib = os.path.abspath(libs[0])
    shutil.copy2(lib, os.path.join(AI_DIR, "flaw_core.so"))
    shutil.copy2(lib, os.path.join(os.path.dirname(AI_DIR), "flaw_core.so"))
    print("\n✅ BUILD READY!")
else:
    print("\n❌ BUILD FAILED or AI folder missing!")

### 🏁 Step 3: Run
Start training.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"
import sys, os

# ROBUST PATH DISCOVERY
PROJECT_ROOT = None
for r, dirs, files in os.walk('/content/flaw_worker'):
    low_dirs = [d.lower() for d in dirs]
    if 'ai' in low_dirs and 'core' in low_dirs:
        PROJECT_ROOT = r
        break

if not PROJECT_ROOT:
    print("Trying secondary discovery...")
    for r, dirs, files in os.walk('/content/flaw_worker'):
        for f in files:
            if f.lower() == 'selfplay_worker.py':
                PROJECT_ROOT = os.path.dirname(r)
                break

if not PROJECT_ROOT:
    print("❌ FAIL: Could not find project folders!")
    print("Dumping root content:")
    !find /content/flaw_worker -maxdepth 3
else:
    print(f"✅ Project Root: {PROJECT_ROOT}")
    if PROJECT_ROOT not in sys.path: sys.path.insert(0, PROJECT_ROOT)
    
    low_dirs = [d.lower() for d in os.listdir(PROJECT_ROOT)]
    if 'ai' in low_dirs:
        real_ai_name = os.listdir(PROJECT_ROOT)[low_dirs.index('ai')]
        init_file = os.path.join(PROJECT_ROOT, real_ai_name, "__init__.py")
        if not os.path.exists(init_file):
            print(f"Creating missing {real_ai_name}/__init__.py...")
            with open(init_file, 'w') as f: pass

    try: 
        from ai.selfplay_worker import DistributedWorker
        print("🚀 Starting worker...")
        worker = DistributedWorker(MASTER_URL, depth=1)
        worker.run()
    except Exception as e:
        print(f"❌ ERROR: {e}")
        !find /content/flaw_worker -maxdepth 3